In [1]:
# imports
# from collections import Counter
# from itertools import islice
import re
import pandas as pd
import string
import csv
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import math
import operator
import numpy as np
porter = PorterStemmer()
# #########################
# DATA CLEANING TECHNIQUES
# #########################
# split by sentance
# remove puncuation
# lowercase letters
# remove/convert numbers
# -------------------------

# #########################
# NLP TECHNIQUES
# #########################
# stemming
# -------------------------
# sent1 = "i am sam"
# sent2 = "sam i am"
# sent3 = "i do not like green eggs and ham"
# sent4 = "the quick person did not realize his speed and the quick person bumped "


# # words = re.findall("\w+", sent4)
# # bigramCount = dict(Counter(zip(words, islice(words, 1, None))))

# # for i in bigramCount.keys():
# #     print(str(i) + " count: %d" % bigramCount[i])

In [ ]:
# @param q: the column name of the query article
# This is when calculating between vectors of the same length
def CosineScore(q):
    scores = {}
    query = weightmatrix.loc[:,q].values
    normq = (query@query)
    for doc in doc_index.keys():
        if not doc == q:
            # divide by the values of the weight vector norms
            d = weightmatrix.loc[:,doc].values
            normd = d@d
            scores[doc] = (query@d)/((normq*normd)**0.5)
    sorted_scores = sorted(scores.items(), key = operator.itemgetter(1), reverse = True)
    return sorted_scores[:3]


In [50]:
stopwords = {"i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "between", "into", "through", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "that", "thats", "that's", "than", "too", "very", "can", "will", "just", "should", "now"}
gibberish = ['\(CNN\)', '\\\\\\\'', '\'', 'Read More', 'CNN', 'Getty Images', '\\\\xc2', '\\\\xb0', '\\\\xc3', '\\\\x94', '\\\\x9c', '\\\\x9d', '\\\\x95', '\\\\xb3', '\\\\xe2', '\\\\x80', '\\\\x99', '\\\\xa9', '\\\\xf0', '\\\\x9f', '\\\\x91']
collection = str.maketrans('', '', string.punctuation)

# load in the crawled data and perform some cleaning
fullnews = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\newsgrams\covidLabeled.csv")
# print(news.shape[0])
fullnews = fullnews[:50]

newsdict = {}
newsdict['headline'] = fullnews.loc[:, 'headline'].values
newsdict['body'] = fullnews.loc[:, 'body'].values
newsdict['covid labels'] = fullnews.loc[:, 'covid labels'].values


news = pd.DataFrame.from_dict(newsdict)
newsdict = ''
# for i in range(news.shape[0]):
#     s = news.iloc[i][3][2:]
#     s2 = news.iloc[i][1][2:]
#     news.at[i,'body'] = s
#     news.at[i,'headline'] = s2

print("done")


for i in range(len(gibberish)):
    for j in range(news.shape[0]):
        output = news.iloc[j][1]
        output = re.sub(gibberish[i], '', output)
        output = re.sub('\\\\n', ' ', output)
        output = re.sub('-', ' ', output)
        news.at[j,'body'] = output

print("############## all clean and ready for cross-val ################")

gibberish = ''
fullnews = ''

done
############## all clean and ready for cross-val ################


In [19]:
def cleantokens(news):
    dterms = {}
    doc_labels = {}
    for j in range(news.shape[0]):
        tokens = word_tokenize(news.iloc[j][1])
        tokens = [t.lower() for t in tokens]
        depunc = [t.translate(collection) for t in tokens]
        words = [w for w in depunc if not w  == '']
        cleaner = [w for w in words if not w in stopwords]
        stemmed = [porter.stem(word) for word in cleaner]
        dterms[str(j)+news.iloc[j][0]] = stemmed
        doc_labels[str(j)+news.iloc[j][0]] = news.iloc[j][2]
    return dterms, doc_labels

In [20]:
#function that creates termlist and doc_idx as we count each doc_length
def populate_data_structs(dterms, term_list, doc_index):
    for doc in dterms.keys():
        term_list += [(word, doc) for word in dterms[doc]]
        doc_index[doc] = len(dterms[doc])

def make_term_index(term_list, term_index):
    def update_term(item):
        i = 0
        for listitem in term_index[item[0]]:
            # if term already found in doc
            if (item[1] == listitem[0]):
                listitem[1] += 1
                return
            else:
                i += 1
        term_index[item[0]].append( [item[1] , 1 ] )
    
    for item in term_list:
        if item[0] not in term_index:
            # add term
            term_index[item[0]] = [[item[1], 1]]
        else:
            # update term
            update_term(item)

In [60]:
def cosine_score(query_words, doc_index, term_index):
    def get_q_weight(term):
        idf = 1 + math.log(len(doc_index)/len(term_index[term])) if term in term_index else 1
        return (query_words.count(term) / len(query_words)) * idf
    
#     def get_d_weight(term, pair):
#         if term in term_index:
#             return (pair[1]/doc_index[pair[0]]) * (1 + math.log(len(doc_index)/len(term_index[term])))
#         else:
#             return 0

    scores = {key: 0 for key in doc_index.keys()}
    for term in query_words:
        for pair in term_index[term] if term in term_index else []:
#             scores[pair[0]] += get_q_weight(term) * get_d_weight(term, pair)
            scores[pair[0]] += get_q_weight(term) * (pair[1]/doc_index[pair[0]]) * (1 + math.log(len(doc_index)/len(term_index[term])))
    scores = {doc: scores[doc] / (doc_index[doc]*len(query_words)) for doc in doc_index.keys()}
    return sorted(scores.items(), key = operator.itemgetter(1), reverse = True)[:5]

In [61]:
splits = 10
pointspersplit = 5
covidlabels = {}
three_nearest_neighbors = {}
errors = []
for i in range(splits):
    end = pointspersplit*(i+1)
    start = end - pointspersplit
    valid = news.iloc[start:end][:]
    train = pd.concat([news.iloc[:start][:], news.iloc[end:][:]])
    dterms, dlabels = cleantokens(train)
    valid_dterms, valid_doc_labels = cleantokens(valid)
    term_list = []
    doc_index = {}
    populate_data_structs(dterms, term_list, doc_index)
    term_index = {}
    make_term_index(term_list, term_index)
    error = 0
    print('Validate %d' % (i+1))
    for q in valid_dterms.keys():
        covidlabels['quarantine'] = 0
        covidlabels['social distancing'] = 0
        covidlabels['treatment'] = 0
        covidlabels['economic effects'] = 0
        covidlabels['pandemic'] = 0
        labels = []
        scores = cosine_score(valid_dterms[q], doc_index, term_index)
        for j in range(len(scores)):
            labels.append(dlabels[scores[j][0]]) # append each label from closest to farthest
            covidlabels[dlabels[scores[j][0]]] += 1
        top2 = sorted(covidlabels.items(), key = operator.itemgetter(1), reverse = True)[:2]
        max_label = ''
        if top2[0][1] == top2[1][1]:
            # pick the closer one
            for l in labels:
                if l == top2[0][0] or l == top2[1][0]:
                    max_label = top2[0][0] if l == top2[0][0] else top2[1][0] 
                    break
        else:
            max_label = top2[0][0]
        if not max_label == valid_doc_labels[q]:
            error += 1
        three_nearest_neighbors[q] = [scores, max_label]
    errors.append(error)
    
print('Average error per validation split: %f%%' % (np.mean(errors)*100/pointspersplit))

Validate 1
Validate 2
Validate 3
Validate 4
Validate 5
Validate 6
Validate 7
Validate 8
Validate 9
Validate 10
Average error per validation split: 50.000000%


In [71]:

# df = pd.DataFrame.from_dict(three_nearest_neighbors, orient='index', columns=['3-NN', 'Label'])
# df.to_csv (r'C:\Users\Rogith\Desktop\Notebooks\newsgrams\ThreeNN.csv', index = True, header=True)

print(len(trainsets))

8


In [ ]:
# assign labels for articles using best matching keyword set 
keywords = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\newsgrams\labelandKeywords.csv")
print(keywords.shape[0])
display(keywords.head())
label_keywords = {}
for j in range(keywords.shape[0]):
    tokens = word_tokenize(keywords.iloc[j][1])
    tokens = [t.lower() for t in tokens]
    cleaner = [w for w in tokens if not w in stopwords]
    stemmed = [porter.stem(word) for word in cleaner]
    label_keywords[keywords.iloc[j][0]] = stemmed
    
dterms, doc_labels = cleantokens(news)
article_covid_labels = []
words_in_labels = {}
q_index = {}
qterm_list = []
qterm_index = {}
populate_data_structs(label_keywords, words_in_labels, qterm_list, q_index)
make_term_index(qterm_list, qterm_index)

for doc in dterms.keys():
    scores = cosine_score(dterms[doc], q_index, qterm_index)
    article_covid_labels.append(scores[0][0])

news['covid labels']  = article_covid_labels

# news.to_csv (r'C:\Users\Rogith\Desktop\Notebooks\newsgrams\covidLabeld.csv', index = False, header=True)
    

In [ ]:
def termweight(term, docno):
    if term in words_in_doc[docno]:
        for postings in term_index[term]:
            if postings[0] == docno:
                return postings[1]/doc_index[postings[0]] * math.log(len(doc_index)/len(term_index[term]), 2)
    return 0


term_doc_matrix = {}

for term in term_index.keys():
    row = []
    for doc in doc_index.keys():
        row.append(termweight(term, doc))
    term_doc_matrix[term] = row

# keys = list(doc_index.keys())
# df = pd.DataFrame.from_dict(term_doc_matrix, orient='index', columns=keys)
# df.to_csv (r'C:\Users\Rogith\Desktop\Notebooks\newsgrams\TFIDFTrainSet.csv', index = True, header=True)

term_list = []
# doc_index = {}
# term_index = {}
words_per_doc = {}
# term_doc_matrix = {}
